# Upload tower stimuli to s3

In [35]:
#Which experiment? bucket_name is the name of the experiment and will be name of the databases both on mongoDB and S3
bucket_name = 'gestalt-static-images' #containment
stim_version = 'Image'

In [36]:
import os
from glob import glob
import boto3
import botocore
from IPython.display import clear_output
import json
import pandas as pd
from PIL import Image

In [37]:
def list_files(paths, ext='png'):
    """Pass list of folders if there are stimuli in multiple folders. 
    Make sure that the containing folder is informative, as the rest of the path is ignored in naming. 
    Also returns filenames as uploaded to S3"""
    if type(paths) is not list:
        paths = [paths]
    results = []
    names = []
    for path in paths:
        results += [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        names += [os.path.split(y)[1] for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    return results,names

In [38]:
## helper to speed things up by not uploading images if they already exist, can be overriden 
def check_exists(s3, bucket_name, stim_name):
    try:
        s3.Object(bucket_name,stim_name).load()    
        return True
    except botocore.exceptions.ClientError as e:    
        if (e.response['Error']['Code'] == "404"):
            print('The object does not exist.')
            return False
        else:
            print('Something else has gone wrong with {}'.format(stim_name))

Pass list of folders if there are stimuli in multiple folders. Make sure that the containing folder is informative, as the rest of the path is ignored in naming.

In [41]:
## set up paths, etc.
# paths_to_stim = ['./example'] ## provide a list of full paths here.
paths_to_stim = [
    
    # tower silhouettes
    "/Users/yoni/Projects/CommonFate/scenes/voronoi/superquadric_4/scene_004/images/"
    
]

# paths_to_stim = glob.glob()
full_stim_paths, filenames = [x for x in list_files(paths_to_stim) if x !='.DS_Store'] #generate filenames and stimpaths
full_stim_paths.sort()
filenames.sort()
print('We have {} stimuli to upload.'.format(len(full_stim_paths)))

We have 64 stimuli to upload.


Upload to S3. This expects the `.aws/credentials` file in your home directory.

In [42]:
reallyRun = True
if reallyRun:

    ## establish connection to s3 
    s3 = boto3.resource('s3')

    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=bucket_name) 
        print('Created new bucket.')
#     except NoCredentialsError:
#         print("Credential missing") #.aws/credentials should be in home folder, not in repo folder
    except:
        b = s3.Bucket(bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = True
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_stim_paths):        # use sorted(full_stim_paths) when not using photodraw32
        stim_name = filenames[i]
        if ((check_exists(s3, bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} as {} | {} of {}'.format(os.path.split(path_to_file)[-1],stim_name,(i+1),len(full_stim_paths)))
            s3.Object(bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(os.path.split(path_to_file)[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)
print('Done!')

Done!


In [43]:
for my_bucket_object in b.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0001.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0002.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0003.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0004.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0005.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0006.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0007.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0008.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0009.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0010.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0011.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0012.png')
s3.ObjectSummary(bucket_name='gestalt-static-images', key='Image0013.png')
s3.ObjectSummary(bucket_n

In [34]:
### add burn-in trials

In [35]:
### add catch trials